<a href="https://colab.research.google.com/github/mjksill/CP540-online/blob/main/notebooks/Pandas_3_Importing_Data_from_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing data into Pandas directly from websites

This worksheet will look at using pandas to import data from websites where csv have been uploaded, and build further on the manipulation of data we seen in the previous worksheet. Today we will use data from the [Open Power Systems Data site](https://data.open-power-system-data.org/renewable_power_plants/), which has multiple CSV files for different countries showing their renewable energy power power plants and subsequent capacity.  If you hover over the links for each country, you will see that there is a URL associated with each CSV file.  Instead of downloading the CSV file from the website and then uploading it into Python, we can directly import the data into a Pandas data frame using the URL:

In [1]:
import pandas as pd
import io

All_countries = {'Germany' : 'DE',                          # We can create a dictionary for all the possible countries
                 'Denmark' : 'DK',
                 'France' : 'FR',
                 'Poland' : 'PL',
                 'United Kingdom' : 'UK',
                 'Switzerland' : 'CH',
                 'Sweden' : 'SE',
                 'Czech Republic' : 'CZ'}

Chosen_country = 'United Kingdom'                          # We can then define the country we want to look at
Country = All_countries[Chosen_country]                    # This assigns the country code, i.e. 'UK' to Country

URL = f'https://data.open-power-system-data.org/renewable_power_plants/2020-08-25/renewable_power_plants_{Country}.csv'
                                                            # This can then be used to pull the specific file we want from the website

df = pd.read_csv(URL, lineterminator='\n', skipinitialspace=True)
print(df)
print(df.columns)


      electrical_capacity energy_source_level_1 energy_source_level_2  \
0                     9.9      Renewable energy                 Hydro   
1                    19.5      Renewable energy                 Hydro   
2                    65.0      Renewable energy                 Hydro   
3                    32.0      Renewable energy                 Hydro   
4                    15.0      Renewable energy                 Hydro   
...                   ...                   ...                   ...   
2615                  0.5      Renewable energy                 Solar   
2616                  0.2      Renewable energy                 Solar   
2617                  0.4      Renewable energy                 Solar   
2618                  0.5      Renewable energy                 Solar   
2619                  5.0      Renewable energy                 Solar   

     energy_source_level_3                       technology data_source  \
0                      NaN  Other or unspecified

As we can see this is a large file with many columns that are either not important to us, or not filled with any data. Thus we should look to remove them. Looking at all the headings when opening the csv file, the only columns we want to keep are 'electrical_capacity', 'energy_source_level_2', 'region' and 'country'. Due to `pandas` indexing, these correspond to columns 0, 2, 14 and 15. So we can go ahead and remove all the other unrequired columns.

In [ ]:
df = df.drop(df.columns[[1,3,4,5,6,7,8,9,10,11,12,13,16,17,18,19,20,21,22,23,24]], axis=1)

print(df)

Now we need to find all the regions where renewable energy plants are stationed in the UK. This can be done by collecting all the unique regions mentioned in the column and collecting them in a list.

In [ ]:
region = df['region'].unique().tolist()                        # This collects all different regions in the column 'region', removes any repeated elements,
                                                               # and puts the result in a list for us to use
print(region)

But wait. At the end of the list we can see there is a region called 'nan'. This is not a region, it is what pandas puts at a point in the dataframe when there is missing or no data in the corresponding cell in the csv file. We could leave this in, but it's untidy, so it is better to remove it.

In [ ]:
df = df.dropna()                                            # This command will remove rows that are missing data in one or more columns

places = df['region'].unique().tolist()                        # We can create a new list, where 'nan' isn't included
print(places)

Say we now want to find out how much electrical capacity there is in each region of the UK from these renewable energy plants.  We can use a similar method to that used last time when we were calculating the solar energy produced.

In [ ]:
import pylab as plt
import numpy as np

sumcapacity = []                                            # Create an empty list to collect the summed capacity in

for i in places:
   sumcapacity.append((df.loc[df['region'] == i, 'electrical_capacity'].sum()))  # This makes use of the Boolean principles,
                                                                              # It will sum the electrical capacity and deposit in the list created

print(sumcapacity)

height = sumcapacity                                        # We can then plot a bar graph to show this data
bars = places
y_pos = np.arange(len(bars))

# Create bars
plt.bar(y_pos, height)

# Create names on the x-axis
plt.xticks(y_pos, bars)
plt.xticks(rotation=90)

plt.xlabel('Regions')
plt.ylabel('Electrical Capacity (MW)')

# Show graphic
plt.show()

We can again use a pie chart to show the share of the UK's capacity between regions.

In [ ]:
# Create a pieplot
plt.pie(sumcapacity, labels=places, labeldistance=1.15);

plt.show()

We can also apply filters to extract specific data from the dataframe. So for example, we can filter the data down to only renewable solar sources in Scotland, and find the total capacity for solar energy.

In [ ]:
is_scotland =  df['region']=='Scotland'                     # All the rows which contain 'Scotland' in the region column are collected


df_scotland = df[is_scotland]                               # And a new dataframe is made which contains only these rows
print(df_scotland)

is_solar = df_scotland['energy_source_level_2']=='Solar'    # We repeat this process, although this time we collect all the rows which contain 'solar' in the source column
df_scot_solar = df_scotland[is_solar]                       # And a new dataframe is created again
print(df_scot_solar)

Total = df_scot_solar['electrical_capacity'].sum()          # We can then sum the values in the second data frame created to find a total value for solar capacity
print(f'Scotlands electrical capacity from solar sources is {Total} MW')    # And then print this on the sheet


The fact that we have written a program to perform the calculation for Scotland's electric capacity from solar sources, allows us to easily perform the same calculation for any other of Scotland's renewable energy sources.  In fact, we can construct a pie chart to see the relative proportions of each type of renewable.

In [ ]:
print( df_scotland['energy_source_level_2'].unique().tolist() )

renewable_list = df_scotland['energy_source_level_2'].unique().tolist()

data_list = []
for renewable in renewable_list:
  check = df_scotland['energy_source_level_2'] == renewable
  tmp = df_scotland[check]
  total = tmp['electrical_capacity'].sum()
  data_list.append(total)
  print(f'{renewable}: {total:.1f} MW')

plt.pie(data_list, labels=renewable_list)
plt.title('Scotland')
plt.show()

In [ ]:
print( df_scotland['energy_source_level_2'].unique().tolist() )

renewable_list = df_scotland['energy_source_level_2'].unique().tolist()

data_list = []
for renewable in renewable_list:
  check = df_scotland['energy_source_level_2'] == renewable
  tmp = df_scotland[check]
  total = tmp['electrical_capacity'].sum()
  data_list.append(total)
  print(f'{renewable}: {total:.1f} MW')

plt.bar(renewable_list, data_list)
plt.title('Scotland')
plt.ylabel('capacity / MW')
plt.show()

In fact, we can repeat the entire process in a fairly simple manner for any country that we wish.

In [ ]:
import pandas as pd
import io

All_countries = {'Germany' : 'DE',                          # We can create a dictionary for all the possible countries
                 'Denmark' : 'DK',
                 'France' : 'FR',
                 'Poland' : 'PL',
                 'United Kingdom' : 'UK',
                 'Switzerland' : 'CH',
                 'Sweden' : 'SE',
                 'Czech Republic' : 'CZ'}

Chosen_country = 'Germany'                          # We can then define the country we want to look at
Country = All_countries[Choosen_country]                    # This assigns the country code, i.e. 'UK' to Country

URL = f'https://data.open-power-system-data.org/renewable_power_plants/2020-08-25/renewable_power_plants_{Country}.csv'
                                                            # This can then be used to pull the specific file we want from the website

df = pd.read_csv(URL, lineterminator='\n', skipinitialspace=True)

renewable_list = df['energy_source_level_2'].unique().tolist()

data_list = []
for renewable in renewable_list:
  check = df['energy_source_level_2'] == renewable
  tmp = df[check]
  total = tmp['electrical_capacity'].sum()
  data_list.append(total)
  print(f'{renewable}: {total:.1f} MW')

plt.bar(renewable_list, data_list)
plt.title(Chosen_country)
plt.ylabel('capacity / MW')
plt.show()

In fact, we can put the entire process in a loop to repeat the entire analysis for all the countries available in the website.

In [ ]:
import pandas as pd
import io

All_countries = {'Germany' : 'DE',                          # We can create a dictionary for all the possible countries
                 'Denmark' : 'DK',
                 'France' : 'FR',
                 'Poland' : 'PL',
                 'United Kingdom' : 'UK',
                 'Switzerland' : 'CH',
                 'Sweden' : 'SE',
                 'Czech Republic' : 'CZ'}

for key, Country in All_countries.items():

  print(key)
  URL = f'https://data.open-power-system-data.org/renewable_power_plants/2020-08-25/renewable_power_plants_{Country}.csv'
  df = pd.read_csv(URL, lineterminator='\n', skipinitialspace=True)

  renewable_list = df['energy_source_level_2'].unique().tolist()

  data_list = []
  for renewable in renewable_list:
    check = df['energy_source_level_2'] == renewable
    tmp = df[check]
    total = tmp['electrical_capacity'].sum()
    data_list.append(total)
    print(f'{renewable}: {total:.1f} MW')

  plt.bar(renewable_list, data_list)
  plt.title(key)
  plt.ylabel('capacity / MW')
  plt.show()

### Conclusion

In this notebook, we have seen how to import data directly from a CSV file located on a website.  We have also learned more about filtering manipulating data frames to determine the results we want.  Writing the processing steps in code allowed us to easily repeat any analysis on any data set.